In [23]:
# Import necessary libraries
import splitfolders
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Split dataset into train (70%), val (20%), test (10%) with stratified sampling
original_dataset_dir = "./dataset"
output_dir = "./datasplit"

splitfolders.ratio(
    original_dataset_dir,
    output=output_dir,
    seed=42,
    ratio=(0.7, 0.2, 0.1),
    move=False,
)

# Define paths
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')
test_dir = os.path.join(output_dir, 'test')

# Image dimensions and parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode="nearest",
)

# Rescale for validation and testing
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)




Found 483 images belonging to 4 classes.
Found 137 images belonging to 4 classes.
Found 74 images belonging to 4 classes.


In [26]:
# Load EfficientNetB0 as base model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Build the model
model = Model(inputs=base_model.input, outputs=predictions)

In [28]:

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train the model with frozen base layers
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
)



Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 76s 3s/step - accuracy: 0.2495 - loss: 1.4369 - val_accuracy: 0.2555 - val_loss: 1.3909 - learning_rate: 1.0000e-04
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.2628 - loss: 1.4007 - val_accuracy: 0.2555 - val_loss: 1.3945 - learning_rate: 1.0000e-04
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.2357 - loss: 1.4177 - val_accuracy: 0.2555 - val_loss: 1.3879 - learning_rate: 1.0000e-04
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.2797 - loss: 1.4045 - val_accuracy: 0.2555 - val_loss: 1.3882 - learning_rate: 1.0000e-04
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.2735 - loss: 1.3973 - val_accuracy: 0.2555 - val_loss: 1.3884 - learning_rate: 1.0000e-04
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.2517 - loss: 1.4004 - val_accuracy: 0.2555 - val_loss: 1.3890 - learning_rate: 1.0000e-04
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.2834 - loss:

In [29]:


# Fine-tune the model
base_model.trainable = True

# Gradually unfreeze the layers
for layer in base_model.layers[:100]:
    layer.trainable = False

# Re-compile with lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5), 
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
history_fine_tune = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 100s 3s/step - accuracy: 0.2343 - loss: 1.5320 - val_accuracy: 0.2555 - val_loss: 1.3874 - learning_rate: 1.0000e-05
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.2479 - loss: 1.4952 - val_accuracy: 0.2555 - val_loss: 1.3870 - learning_rate: 1.0000e-05
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.2506 - loss: 1.4960 - val_accuracy: 0.2482 - val_loss: 1.3875 - learning_rate: 1.0000e-05
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.2613 - loss: 1.4376 - val_accuracy: 0.2482 - val_loss: 1.3885 - learning_rate: 1.0000e-05
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.2940 - loss: 1.4248 - val_accuracy: 0.2482 - val_loss: 1.3887 - learning_rate: 1.0000e-05
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 41s 3s/step - accuracy: 0.3323 - loss: 1.4202 - val_accuracy: 0.2409 - val_loss: 1.3888 - learning_rate: 2.0000e-06
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.2962 - loss

In [30]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc:.2f}")

# Classification report for further analysis
from sklearn.metrics import classification_report

y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=list(test_generator.class_indices.keys())))

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 576ms/step - accuracy: 0.1635 - loss: 1.3890
Test Accuracy: 0.26
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 4s/step 
              precision    recall  f1-score   support

       Angry       0.00      0.00      0.00        18
        Fear       0.00      0.00      0.00        19
       Happy       0.00      0.00      0.00        18
         Sad       0.26      1.00      0.41        19

    accuracy                           0.26        74
   macro avg       0.06      0.25      0.10        74
weighted avg       0.07      0.26      0.10        74



c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,